In [64]:
import pandas as pd

# Reading in the MoMA collection csv and viewing the first five rows. 
df_moma = pd.read_csv("../data/raw/artworks.csv")
df_moma.head(5)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,OnView,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,NaN,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(male),1980,Photographic reproduction with colored synthet...,...,NaN,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,NaN,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


I will start by removing unnecessary columns.

In [65]:
df_moma = df_moma.drop(columns=['ConstituentID','ArtistBio','Nationality','BeginDate','EndDate','Date','Medium','Dimensions','CreditLine','AccessionNumber','Classification','Department','Cataloged','ObjectID','URL','ImageURL','OnView','Circumference (cm)','Depth (cm)','Diameter (cm)','Height (cm)','Length (cm)','Weight (kg)','Width (cm)','Seat Height (cm)','Duration (sec.)'])
     

In [66]:
#Viewing updated dataset with unnecessary columns removed.
df_moma.head(5)

,Title,Artist,Gender,DateAcquired
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,(male),1996-04-09
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,(male),1995-01-17
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,(male),1997-01-15
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,(male),1995-01-17
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,(male),1997-01-15


Next, I will rename the columns.

In [67]:
df_moma = df_moma.rename(columns={'Title':'title','Artist':'artist','Gender':'gender','DateAcquired':'date_acquired'})
df_moma.head(5)

,title,artist,gender,date_acquired
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,(male),1996-04-09
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,(male),1995-01-17
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,(male),1997-01-15
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,(male),1995-01-17
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,(male),1997-01-15


Next, I will change title, artist, and gender to string data types.

In [68]:
df_moma.title = df_moma.title.astype('string')
df_moma.artist = df_moma.artist.astype('string')
df_moma.gender = df_moma.gender.astype('string')


In [69]:
#Checking to see if data types were updated. 
df_moma.dtypes

title            string[python]
artist           string[python]
gender           string[python]
date_acquired            object
dtype: object

Next, I will change the "date" column to a datetime data type.

In [70]:
df_moma['date_acquired'] = pd.to_datetime(df_moma['date_acquired'])

In [71]:
df_moma.dtypes

title            string[python]
artist           string[python]
gender           string[python]
date_acquired    datetime64[ns]
dtype: object

Next, I will filter rows with missing artist names, gender and acquisition dates because these values are needed for my analysis. First, I will check for empty fields.  

In [72]:
df_moma.isnull().sum()

title              48
artist           1260
gender           1260
date_acquired    6782
dtype: int64

I will start by dropping rows with empty fields. 

In [73]:
df_moma.dropna(inplace=True)

In [74]:
#Checking to see if rows with empty fields were dropped. 
df_moma.isnull().sum()

title            0
artist           0
gender           0
date_acquired    0
dtype: int64

Next, I will clean the gender column. As noted during the discovery process, multiple artists can be listed for one art piece. Furthermore, "()" is sometimes used for unknown genders. 

In [75]:
df_moma['gender'].value_counts().to_frame()

,count
gender,
(male),114911
(female),18306
(),7426
(male) (male),1670
(male) (male) (male),1059
...,...
() () (male) (male) (),1
(male) (male) (male) (male) (male) (female) (female) (male) (male) (male) (male) (male) (female) (male) (male) (male) (male) (male) (female) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (female) (female) (male) (male) (male) (male) (male) (female) (female) (male) (female) (male) (male) (male) (female) (male) (male) (male) (male) (male) (male) (female) (female) (female) (male) (male) (female) (male) (male) (male) (female) (male) (male) (male) (male) (male) (male) (male) (female) (female) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (female) (male) (male) (female) (male) (male) (male) (female) (male) (male) (female) (female) (male) (female) (male) (female) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (female) (),1
(transgender woman),1


I will need to separate the multiple artists and remove the "()"s. I will attempt this using the explode function. However, first I need to address the unclear gender listings in the array above. Upon review of the raw data "(unknown. (non-binary or trans?))" was used for an artist named Toyen who is known for being non-binary. First, I will replace "(unknown. (non-binary or trans?))" with "non-binary".

In [76]:
df_moma['gender'] = df_moma['gender'].astype(str).replace('(unknown. (non-binary or trans?))', '(non-binary)')

This replaced "(unknown. (non-binary or trans?))" with "non-binary" when listed as a single value, but not when listed with other genders. I will attempt to remedy this for this specific multiple artist listing.

In [77]:
df_moma['gender'] = df_moma['gender'].astype(str).replace('(male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (female) (male) (male) (male) (female) (male) (male) (female) (male) (female) (unknown. (non-binary or trans?)) (female) (male) (male) ()', '(male) (male) (male) (male) (male) (male) (male) (male) (male) (male) (female) (male) (male) (male) (female) (male) (male) (female) (male) (female) (non-binary) (female) (male) (male) ()')

Next, I will simplify "(transgender','woman)" as "transgender".  

In [78]:
df_moma['gender'] = df_moma['gender'].astype(str).replace('(transgender woman)', '(transgender)')

Next, I will simplify "(gender non-conforming)" to "non-conforming".

In [79]:
df_moma['gender'] = df_moma['gender'].astype(str).replace('(gender non-conforming)', '(non-conforming)')

Next, I will simplify "(female (transwoman))" to "transgender". 

In [80]:
df_moma['gender'] = df_moma['gender'].astype(str).replace('(female (transwoman))', '(transgender)')

In [81]:
#Checking to see if the gender listings were updated. 
df_moma.gender.unique()

array(['(male)', '(male) (male)', '(male) (female)',
       '(male) (male) (male)', '(female) (male) ()', '(female)',
       '(male) (female) (male) (female)', '() (male) (male)',
       '(male) (male) () (female)', '() (male) (male) (female)',
       '(male) (male) ()', '(male) (male) (female) (female)',
       '() (male) (female)', '(male) (female) (female) (male)',
       '(male) () (male) (female) (male) (male) (male) (male) (female)',
       '() (male)', '(male) ()', '()',
       '() (male) (male) (male) (male) (male) (male)', '(male) () ()',
       '() (male) (male) (male) (male) (male)', '(female) (male)',
       '() (male) (male) (male) (male) (male) (male) (male)',
       '() (male) (male) (male) (female) (male)',
       '(male) (male) (male) (female) (male) (male) (male) (male) (male) (male)',
       '(male) (male) (male) (male) (female) (male)',
       '(male) (female) (male)', '(male) (male) () ()',
       '(male) (male) (male) (male) (male) (male)',
       '(male) () (male

Next, I will use the explode function to separate multiple artists for one art piece into separate rows. In addition, I will remove the "()"s for unknown genders. 

In [82]:
# The multiple artists are not separated by commas. Therefore, I will have to split them by the spaces between the genders. Gender_list will be the new gender column.
df_moma['gender_list'] = df_moma['gender'].str.split()  
# The explode function will put the artists into separate rows.
df_moma = df_moma.explode('gender_list')  
# Now I will remove the unknown genders marked as "()".
df_moma = df_moma[df_moma['gender_list'].str.strip() != '()']  


In [83]:
#Checking the gender numbers prior to explode.
df_moma['gender'].value_counts().to_frame()

,count
gender,
(male),114911
(female),18306
(male) (male),3340
(male) (male) (male),3177
(male) (female),1648
...,...
() () () () () (male) (),1
() (male) () () (),1
(male) () () () () () () () () () () () () () () (),1


In [84]:
#Checking the new numbers after explode. 
df_moma['gender_list'].value_counts().to_frame()

,count
gender_list,
(male),130515
(female),21943
(transgender),63
(non-binary),17
(non-conforming),2


Now that I have my new "gender_list" column, I will remove the old "gender" column to simplify the data set. 

In [85]:
df_moma = df_moma.drop(columns=['gender'])

In [86]:
#Checking columns for new data set. 
df_moma.head(5)

,title,artist,date_acquired,gender_list
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,1996-04-09,(male)
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,1995-01-17,(male)
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,1997-01-15,(male)
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,1995-01-17,(male)
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,1997-01-15,(male)


In [87]:
#Checking data set info.
df_moma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 152540 entries, 0 to 155913
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   title          152540 non-null  string        
 1   artist         152540 non-null  string        
 2   date_acquired  152540 non-null  datetime64[ns]
 3   gender_list    152540 non-null  object        
dtypes: datetime64[ns](1), object(1), string(2)
memory usage: 5.8+ MB


I will change the new "gender_list" column to a string data type. 

In [88]:
df_moma.gender_list = df_moma.gender_list.astype('string')

In [89]:
#Checking to see if data type for "gender_list" was updated.
df_moma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 152540 entries, 0 to 155913
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   title          152540 non-null  string        
 1   artist         152540 non-null  string        
 2   date_acquired  152540 non-null  datetime64[ns]
 3   gender_list    152540 non-null  string        
dtypes: datetime64[ns](1), string(3)
memory usage: 5.8 MB


In [90]:
#Writing csv to clean data folder.
df_moma.to_csv('../data/clean/clean_artworks.csv', index=False)
